# Chapter 17: PEFT OSF를 활용한 금융 도메인 지속 학습

## 학습 목표
- OSF(Orthogonal Subspace Fine-tuning)의 원리와 장점을 이해한다
- HuggingFace PEFT 라이브러리를 활용하여 OSF를 구현한다
- 금융 도메인 데이터셋으로 지속 학습을 수행한다
- 재앙적 망각 없이 새로운 도메인 지식을 주입하는 방법을 학습한다

---

## 1. OSF (Orthogonal Subspace Fine-tuning) 개요

### 1.1 지속 학습의 과제: 재앙적 망각 (Catastrophic Forgetting)

LLM을 새로운 태스크에 적응시킬 때 가장 큰 문제는 **재앙적 망각**이다. 새로운 데이터를 학습하면 이전에 학습한 지식이 덮어써지는 현상이다.


### 1.2 OSF란?

OSF(Orthogonal Subspace Fine-tuning)는 2024년 논문에서 제안된 PEFT 기법으로, **지속 학습(Continual Learning)**에 특화되어 있다. 핵심 아이디어는 **SVD(특이값 분해)**를 활용하여 가중치 행렬을 분해하고, 기존 지식을 담당하는 부분은 동결하고 나머지 부분만 학습하는 것이다.

### 1.3 OSF 작동 원리

OSF는 각 가중치 행렬을 SVD로 분해한다:

```
W = U_high × S_high × V_high^T  +  U_low × S_low × V_low^T
    ─────────────────────────      ───────────────────────
           동결 (Frozen)                학습 (Trainable)
           기존 지식 보존                 새로운 태스크 적응
```


### 1.4 OSF vs LoRA 비교

| 특성 | LoRA | OSF |
|------|------|-----|
| **목적** | 파라미터 효율적 학습 | 지속 학습 (망각 방지) |
| **추가 파라미터** | 있음 (A, B 행렬) | 없음 (in-place 수정) |
| **rank의 의미** | 학습 가능한 rank | 보존할 rank (frozen) |
| **망각 방지** | 제한적 | 강력함 |
| **순차 학습** | 어려움 | 설계 목적 |

### 1.5 핵심 파라미터: effective_rank

OSF에서 `effective_rank`는 **보존할(동결할) rank**를 의미한다. LoRA의 `r`과 반대 개념이다.

| effective_rank | 의미 | 권장 용도 |
|----------------|------|----------|
| **높은 값** | 많은 부분 동결 | 기존 지식 최대 보존 |
| **낮은 값** | 적은 부분 동결 | 새 지식 학습 우선 |
| **None** | 자동 (50%) | 기본값, 균형 |
| **0.5 (분수)** | 50% 동결 | 비율로 지정 |

## 2. 환경 설정

In [ ]:
# PEFT 0.18.0 이상 필요 (OSF 지원)
%pip install -q peft>=0.18.0 transformers datasets accelerate bitsandbytes

In [16]:
import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import OSFConfig, get_peft_model

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 경로 설정
BASE_PATH = "./OSFT_Finance_Practice"
MODEL_PATH = "./GPT-OSS-20B-Merged"  # 테스트용 (실제: 더 큰 모델 사용 가능)
OUTPUT_DIR = os.path.join(BASE_PATH, "checkpoints")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"\n출력 경로: {OUTPUT_DIR}")

PyTorch 버전: 2.6.0+cu126
CUDA 사용 가능: True
GPU: NVIDIA H100 80GB HBM3
VRAM: 79.2 GB

출력 경로: ./OSFT_Finance_Practice/checkpoints


## 3. 금융 데이터셋 로드 및 전처리

### 도메인별 데이터셋

In [17]:
# 도메인별 대표 데이터셋 (참고용)
domain_datasets = {
    "금융": "gbharti/finance-alpaca",
    "의료": "medalpaca/medical_meadow_medical_flashcards",
    "코딩": "TokenBender/code_instructions_122k_alpaca_style"
}

# 1. 금융 데이터셋 로드
print("금융 데이터셋 로딩 중...")
dataset = load_dataset("gbharti/finance-alpaca", split="train[:1000]")

print(f"데이터 개수: {len(dataset)}")
print(f"컬럼: {dataset.column_names}")
print(f"\n샘플 데이터:")
print(json.dumps(dataset[0], indent=2, ensure_ascii=False))

금융 데이터셋 로딩 중...
데이터 개수: 1000
컬럼: ['instruction', 'input', 'output', 'text']

샘플 데이터:
{
  "instruction": "For a car, what scams can be plotted with 0% financing vs rebate?",
  "input": "",
  "output": "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0

### 도메인 인스트럭션

In [18]:
def format_domain_instruction(example, domain="금융"):
    """
    도메인별 시스템 프롬프트를 적용하여 데이터를 포맷팅하는 함수다.
    messages 형식으로 변환한다.
    """
    # 도메인별 시스템 프롬프트 정의
    system_prompts = {
        "금융": "당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.",
        "의료": "당신은 의료 정보 AI 어시스턴트이다. 의학적 사실에 기반하여 답변하되, 반드시 전문가와의 상담을 권유하라.",
        "법률": "당신은 법률 전문가 AI 어시스턴트이다. 관련 법령과 판례에 기반하여 답변하되, 법적 효력이 없음을 명시하라."
    }
    
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']
    
    # 해당 도메인의 시스템 프롬프트 선택
    sys_prompt = system_prompts.get(domain, "당신은 유용한 AI 어시스턴트이다.")
    
    # 사용자 메시지 구성
    if input_text:
        user_content = f"{instruction}\n\n입력: {input_text}"
    else:
        user_content = instruction
    
    # messages 형식으로 변환
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": output}
    ]
    
    return {"messages": messages}

# 금융 도메인으로 포맷팅 적용
formatted_dataset = dataset.map(lambda x: format_domain_instruction(x, "금융"))

print("\n[포맷팅된 데이터 샘플]")
print(json.dumps(formatted_dataset[0]['messages'], indent=2, ensure_ascii=False))


[포맷팅된 데이터 샘플]
[
  {
    "content": "당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.",
    "role": "system"
  },
  {
    "content": "For a car, what scams can be plotted with 0% financing vs rebate?",
    "role": "user"
  },
  {
    "content": "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some 

### 데이터 포맷팅 함수 정의

In [19]:
def format_domain_instruction_text(example, domain="금융"):
    """
    도메인별 시스템 프롬프트를 적용하여 텍스트 형식으로 포맷팅하는 함수다.
    (대안 포맷: text 필드 사용)
    """
    # 도메인별 시스템 프롬프트 정의
    system_prompts = {
        "금융": "당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.",
        "의료": "당신은 의료 정보 AI 어시스턴트이다. 의학적 사실에 기반하여 답변하되, 반드시 전문가와의 상담을 권유하라.",
        "법률": "당신은 법률 전문가 AI 어시스턴트이다. 관련 법령과 판례에 기반하여 답변하되, 법적 효력이 없음을 명시하라."
    }
    
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']
    
    # 해당 도메인의 시스템 프롬프트 선택
    sys_prompt = system_prompts.get(domain, "당신은 유용한 AI 어시스턴트이다.")
    
    # 프롬프트 구성
    if input_text:
        text = f"""### System:
{sys_prompt}

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### System:
{sys_prompt}

### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": text}

# 텍스트 형식으로도 포맷팅
formatted_dataset_text = dataset.map(lambda x: format_domain_instruction_text(x, "금융"))

print("\n[텍스트 포맷 샘플]")
print(formatted_dataset_text[0]['text'][:800] + "...")


[텍스트 포맷 샘플]
### System:
당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.

### Instruction:
For a car, what scams can be plotted with 0% financing vs rebate?

### Response:
The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They ...


## 4. 토크나이저 및 모델 로드

In [20]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"토크나이저 로드 완료")
print(f"  vocab_size: {tokenizer.vocab_size}")
print(f"  pad_token: {tokenizer.pad_token}")

The tokenizer you are loading from './GPT-OSS-20B-Merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


토크나이저 로드 완료
  vocab_size: 199998
  pad_token: <|endoftext|>


In [21]:
# 모델 로드 (메모리 효율을 위해 bfloat16 사용)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # Flash Attention 이슈 방지
)

print(f"모델 로드 완료")
print(f"  파라미터 수: {model.num_parameters():,}")
print(f"  dtype: {model.dtype}")

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

모델 로드 완료
  파라미터 수: 20,914,757,184
  dtype: torch.bfloat16


## 5. OSF 설정 및 적용

### 5.1 OSFConfig 파라미터 설명

- `target_modules`: OSF를 적용할 모듈 (예: attention layers)
- `effective_rank`: 보존할 rank (동결할 SVD 성분 수). None일 경우 자동으로 50%가 설정된다.
- `rank_pattern`: 모듈별로 rank를 다르게 지정할 때 사용한다.

```python
OSFConfig(
    target_modules=[...],    # OSF를 적용할 모듈 (예: attention layers)
    effective_rank=8,        # 보존할 rank (동결할 SVD 성분 수)
    rank_pattern={...}       # 모듈별 rank 오버라이드
)
```

**effective_rank의 의미:**
- LoRA의 `r`: 학습 가능한 rank
- OSF의 `effective_rank`: 보존할(동결할) rank
- 학습 가능한 rank = min(weight.shape) - effective_rank

In [22]:
# OSF 설정
osf_config = OSFConfig(
    # 타겟 모듈: Attention 레이어
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    
    # effective_rank: 보존할 rank (기존 지식 보존)
    # None으로 설정하면 자동으로 50% 사용
    effective_rank=None,  # 자동 설정 (권장)
    
    # 모듈별 rank 오버라이드 (선택사항)
    rank_pattern={
        "q_proj": 16,   # Query에는 더 많은 rank 보존
        "v_proj": 8,    # Value에는 적은 rank 보존
    }
)

print("OSF 설정:")
print(f"  target_modules: {osf_config.target_modules}")
print(f"  effective_rank: {osf_config.effective_rank} (None = 자동 50%)")

OSF 설정:
  target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj']
  effective_rank: None (None = 자동 50%)


In [23]:
# OSF 적용
model = get_peft_model(model, osf_config)

# 학습 가능한 파라미터 확인
model.print_trainable_parameters()

print("\nOSF 모델 준비 완료!")

trainable params: 782,584,512 || all params: 21,697,341,696 || trainable%: 3.6068

OSF 모델 준비 완료!


## 6. 데이터셋 토큰화

In [24]:
def tokenize_function(examples):
    """
    텍스트 데이터를 토큰화하는 함수다.
    """
    # text 필드 사용
    texts = examples["text"]
    
    # 토큰화
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )
    
    # labels 설정 (Causal LM용)
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# 토큰화 적용
tokenized_dataset = formatted_dataset_text.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset_text.column_names
)

print(f"토큰화 완료")
print(f"  샘플 수: {len(tokenized_dataset)}")
print(f"  컬럼: {tokenized_dataset.column_names}")

토큰화 완료
  샘플 수: 1000
  컬럼: ['input_ids', 'attention_mask', 'labels']


In [25]:
# Train/Eval 분할
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"학습 데이터: {len(train_dataset)}개")
print(f"평가 데이터: {len(eval_dataset)}개")

학습 데이터: 900개
평가 데이터: 100개


## 7. OSF 학습 실행

In [26]:
# W&B 초기화
import wandb
wandb.init(project="osft-finance", name="finance-osft")

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=os.path.join(OUTPUT_DIR, "osf_finance"),
    
    # 학습 파라미터 (OSF 권장: 낮은 학습률)
    learning_rate=1e-5,          # OSF는 1e-5 ~ 1e-4 권장
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    
    # 옵티마이저
    optim="adamw_torch",
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    
    # 정밀도
    bf16=True,
    
    # 로깅
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    
    # 기타
    load_best_model_at_end=True,
    report_to="wandb",  # wandb 사용 시 "wandb"
)

print("학습 설정 완료")
print(f"  학습률: {training_args.learning_rate}")
print(f"  에폭: {training_args.num_train_epochs}")
print(f"  배치 크기: {training_args.per_device_train_batch_size}")
print(f"  그래디언트 누적: {training_args.gradient_accumulation_steps}")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kubwai to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


학습 설정 완료
  학습률: 1e-05
  에폭: 3
  배치 크기: 4
  그래디언트 누적: 4


In [27]:
# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("Trainer 준비 완료")

Trainer 준비 완료


In [28]:
# 학습 실행

print("OSF 금융 도메인 학습 시작")

trainer.train()

OSF 금융 도메인 학습 시작


Step,Training Loss,Validation Loss
50,2.262300,2.314679
100,2.230300,2.296329
150,2.173900,2.295674


TrainOutput(global_step=171, training_loss=2.4028220929597555, metrics={'train_runtime': 707.4398, 'train_samples_per_second': 3.817, 'train_steps_per_second': 0.242, 'total_flos': 1.751628665585664e+17, 'train_loss': 2.4028220929597555, 'epoch': 3.0})

In [29]:
# 모델 저장
save_path = os.path.join(OUTPUT_DIR, "OSFT_Finance_Final")
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"모델 저장 완료: {save_path}")

모델 저장 완료: ./OSFT_Finance_Practice/checkpoints/OSFT_Finance_Final


## 8. 학습된 모델 테스트

In [30]:
def generate_response(model, tokenizer, question, max_new_tokens=200):
    """
    모델 응답을 생성하는 함수다.
    """
    prompt = f"""### System:
    당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.
    
    ### Instruction:
    {question}
    
    ### Response:
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Response 부분만 추출
    answer = response.split("### Response:")[-1].strip()
    
    return answer

print("응답 생성 함수 준비 완료")

응답 생성 함수 준비 완료


### 금융 관련 질문 테스트

In [32]:
finance_questions = [
    "PER(주가수익비율)이 낮다는 것은 무엇을 의미하나?",
    "금리가 오르면 채권 가격은 어떻게 되나?",
    "분산투자의 목적은 무엇인가?",
]

print("[금융 질문 테스트]")

for q in finance_questions:
    print(f"\nQ: {q}")
    answer = generate_response(model, tokenizer, q)
    print(f"A: {answer[:300]}..." if len(answer) > 300 else f"A: {answer}")
    print("-" * 40)

[금융 질문 테스트]

Q: PER(주가수익비율)이 낮다는 것은 무엇을 의미하나?
A: PER(주가수익비율)은 주가를 주당순이익으로 나눈 값이다. PER이 낮다는 것은 주가가 낮다는 것을 의미한다. 하지만 PER이 낮다고 해서 반드시 가치가 낮은 것은 아니다. PER이 낮다는 것은 주가가 낮다는 것과는 별개로 기업이 현재 이익을 내고 있다는 것을 의미한다. PER이 낮다는 것은 주가가 낮다는 것과는 별개로 기업이 현재 이익을 내고 있다는 것을 의미한다. PER이 낮다는 것은 주가가 낮다는 것과는 별개로 기업이 현재 이익을 내고 있다는 것을 의미한다. PER이 낮다는 것은 주가가 낮다는 것과는 별개로 기업이 현재 이익을...
----------------------------------------

Q: 금리가 오르면 채권 가격은 어떻게 되나?
A: 금리와 채권 가격은 반비례 관계이다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 기존에 채권이 발행된 가격이 낮아지므로 채권 가격이 내려간다. 즉, 채권을 사면 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리와 채권 가격은 반비례 관계이다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내려간다. 금리가 오르면 채권 가격은 내...
----------------------------------------

Q: 분산투자의 목적은 무엇인가?
A: 분산투자의 목적은 리스크를 줄이면서 수익을 높이는 것이다.  
     즉, 분산투자를 하면 투자한 종목이 떨어져도 다른 종목이 올라가면 손실을 줄일 수 있다.  
     반대로, 수익을 높이려면 한 종목에만 투자하는 것보다 여러 종목에 투자하는 것이 좋다.  
     하지만 분산투자를 할수록 수익이 늘어나는 것은 아니다. 분산투자를 하면 리스크는 줄어들지만 수익도 

### 반 지식 질문 테스트 (재앙적 망각 확인)

In [34]:
general_questions = [
    "1 + 1 = ?",
    "대한민국의 수도는?",
    "Python에서 리스트의 길이를 구하는 함수는?",
]

print("\n[일반 지식 테스트 - 재앙적 망각 확인]")

for q in general_questions:
    print(f"\nQ: {q}")
    answer = generate_response(model, tokenizer, q)
    print(f"A: {answer[:200]}..." if len(answer) > 200 else f"A: {answer}")
    print("-" * 40)


[일반 지식 테스트 - 재앙적 망각 확인]

Q: 1 + 1 = ?
A: It depends.  Most banks offer you a low rate in order to keep the money in the bank.  If you want a higher rate, you might have to deposit a large amount of money.  If you have a lot of money, you mig...
----------------------------------------

Q: 대한민국의 수도는?
A: 서울입니다.
```
---

## 📚 Reference

- https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification
- https://huggingface.co/blog/how-to-train
- https://huggingface.co/bert-ba...
----------------------------------------

Q: Python에서 리스트의 길이를 구하는 함수는?
A: len(리스트)
```

위 예시처럼 Markdown 형식으로 질문과 답변을 정리하면, 나중에 찾기 쉽고 이해하기 쉬운 문서가 됩니다. 필요하다면, 코드를 포함한 답변을 별도로 파일로 저장해 두는 것도 좋은 방법입니다.

It looks like you're interested in how to store and organize your questions ...
----------------------------------------


## 9. 순차 학습: 다중 도메인 지속 학습

OSF의 진정한 강점은 **여러 태스크를 순차적으로 학습**할 때 발휘된다. 각 태스크 학습 후 SVD를 재계산하여 보존할 공간을 확장한다.

OSF의 진정한 강점은 여러 태스크를 순차적으로 학습할 때 발휘된다. 각 태스크 학습 후 SVD를 재계산하여 보존할 공간을 확장한다. 아래 코드는 순차 학습의 논리를 보여주는 예시이다.

순차 학습 전략:

1. Task 1 (금융): effective_rank=8 (적게 보존, 많이 학습)
2. Task 2 (의료): effective_rank=12 (Task 1 + 의료 지식 보존을 위해 보존 영역 확장)
3. Task 3 (코딩): effective_rank=16 (Task 1, 2 + 코딩 지식 보존)

즉, 점진적으로 보존 공간을 늘려 이전 지식을 보호하는 방식이다.

In [35]:
def sequential_domain_learning_example():
    """
    다중 도메인 순차 학습 예시다.
    금융 → 의료 → 코딩 순서로 학습하며 각 단계에서 OSF를 재적용한다.
    """
    print("=" * 60)
    print("다중 도메인 순차 학습 (OSF)")
    print("=" * 60)
    
    domains = [
        {"name": "금융", "dataset": "gbharti/finance-alpaca", "rank": 8},
        {"name": "의료", "dataset": "medalpaca/medical_meadow_medical_flashcards", "rank": 12},
        {"name": "코딩", "dataset": "TokenBender/code_instructions_122k_alpaca_style", "rank": 16},
    ]
    
    # 초기 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    
    for i, domain in enumerate(domains, 1):
        print(f"\n[{i}/{len(domains)}] {domain['name']} 도메인 학습...")
        
        # OSF 설정 - 보존 rank 점진적 증가
        osf_config = OSFConfig(
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            effective_rank=domain['rank'],  # 태스크마다 보존 rank 증가
        )
        
        # OSF 적용
        model = get_peft_model(base_model, osf_config)
        
        # 학습 수행 (실제 코드에서는 trainer.train() 호출)
        print(f"  effective_rank: {domain['rank']} (보존할 rank)")
        print(f"  학습 가능한 rank: min(weight.shape) - {domain['rank']}")
        
        # 학습 후 base_model 업데이트 (unload)
        base_model = model.unload()  # OSF 제거하고 업데이트된 가중치만 유지
        
        print(f" {domain['name']} 학습 완료")
    
    print("\n" + "=" * 60)
    print("순차 학습 완료!")
    print("최종 모델은 금융 + 의료 + 코딩 지식을 모두 보유한다.")
    print("=" * 60)

# 예시 출력 (실제 실행은 주석 처리)
sequential_domain_learning_example()

print("순차 학습 예시 함수 준비 완료")
print("\n[순차 학습 전략]")
print("1. Task 1: effective_rank=8 (적게 보존, 많이 학습)")
print("2. Task 2: effective_rank=12 (Task 1 + 추가 보존)")
print("3. Task 3: effective_rank=16 (Task 1,2 + 추가 보존)")
print("→ 점진적으로 보존 공간을 늘려 이전 지식을 보호한다.")

다중 도메인 순차 학습 (OSF)


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]


[1/3] 금융 도메인 학습...
  effective_rank: 8 (보존할 rank)
  학습 가능한 rank: min(weight.shape) - 8
 금융 학습 완료

[2/3] 의료 도메인 학습...


/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


  effective_rank: 12 (보존할 rank)
  학습 가능한 rank: min(weight.shape) - 12
 의료 학습 완료

[3/3] 코딩 도메인 학습...
  effective_rank: 16 (보존할 rank)
  학습 가능한 rank: min(weight.shape) - 16
 코딩 학습 완료

순차 학습 완료!
최종 모델은 금융 + 의료 + 코딩 지식을 모두 보유한다.
순차 학습 예시 함수 준비 완료

[순차 학습 전략]
1. Task 1: effective_rank=8 (적게 보존, 많이 학습)
2. Task 2: effective_rank=12 (Task 1 + 추가 보존)
3. Task 3: effective_rank=16 (Task 1,2 + 추가 보존)
→ 점진적으로 보존 공간을 늘려 이전 지식을 보호한다.


## 10. OSF 고급 설정

In [36]:
# 고급 OSF 설정 예시

# 1. 모듈별 다른 rank 설정
advanced_config_1 = OSFConfig(
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    effective_rank=8,  # 기본값
    rank_pattern={
        "q_proj": 16,     # Query에는 더 많은 rank 보존
        "k_proj": 16,     # Key도 중요
        "v_proj": 8,      # Value는 기본
        "o_proj": 8,      # Output도 기본
        "gate_proj": 4,   # MLP는 적게 보존
        "up_proj": 4,
        "down_proj": 4,
    }
)

print("[고급 설정 1: 모듈별 rank 지정]")
print(f"  Attention: q_proj=16, k_proj=16, v_proj=8, o_proj=8")
print(f"  MLP: gate_proj=4, up_proj=4, down_proj=4")

# 2. 분수 비율로 rank 설정
advanced_config_2 = OSFConfig(
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    effective_rank=0.8,  # 80% 보존, 20% 학습
)

print("\n[고급 설정 2: 비율로 rank 지정]")
print(f"  effective_rank=0.8 → 가중치의 80% 보존, 20% 학습")

# 3. 순차 학습용 점진적 설정
n_tasks = 4
max_rank = 64

print(f"\n[고급 설정 3: {n_tasks}개 태스크 순차 학습]")
for task_id in range(n_tasks):
    preserved_fraction = (task_id + 1) / n_tasks
    preserved_rank = int(max_rank * preserved_fraction)
    trainable_fraction = 1 - preserved_fraction
    
    print(f"  Task {task_id + 1}: preserved_rank={preserved_rank} "
          f"({preserved_fraction:.0%} 보존, {trainable_fraction:.0%} 학습)")

[고급 설정 1: 모듈별 rank 지정]
  Attention: q_proj=16, k_proj=16, v_proj=8, o_proj=8
  MLP: gate_proj=4, up_proj=4, down_proj=4

[고급 설정 2: 비율로 rank 지정]
  effective_rank=0.8 → 가중치의 80% 보존, 20% 학습

[고급 설정 3: 4개 태스크 순차 학습]
  Task 1: preserved_rank=16 (25% 보존, 75% 학습)
  Task 2: preserved_rank=32 (50% 보존, 50% 학습)
  Task 3: preserved_rank=48 (75% 보존, 25% 학습)
  Task 4: preserved_rank=64 (100% 보존, 0% 학습)


## 11. 재앙적 망각 평가

In [38]:
def evaluate_catastrophic_forgetting(model, tokenizer):
    """
    재앙적 망각을 평가하는 함수다.
    일반 능력과 도메인 능력을 모두 테스트한다.
    """
    model.eval()
    
    # 일반 능력 테스트
    general_tests = [
        {"question": "1 + 1 = ?", "expected": "2"},
        {"question": "대한민국의 수도는?", "expected": "서울"},
        {"question": "물의 화학식은?", "expected": "H2O"},
        {"question": "태양계에서 가장 큰 행성은?", "expected": "목성"},
    ]
    
    # 금융 능력 테스트
    finance_tests = [
        {"question": "PER이 낮다는 것은 무엇을 의미하나?", "keywords": ["저평가", "저렴", "수익"]},
        {"question": "금리가 오르면 채권 가격은?", "keywords": ["하락", "떨어", "내려"]},
        {"question": "분산투자의 목적은?", "keywords": ["위험", "리스크", "분산"]},
    ]
    
    print("재앙적 망각 평가")
    print("=" * 50)
    
    # 일반 능력 평가
    print("\n[일반 능력 테스트]")
    general_score = 0
    for test in general_tests:
        response = generate_response(model, tokenizer, test["question"], max_new_tokens=50)
        is_correct = test["expected"].lower() in response.lower()
        status = "✓" if is_correct else "✗"
        print(f"  {status} {test['question']} → {response[:50]}...")
        if is_correct:
            general_score += 1
    
    general_accuracy = general_score / len(general_tests)
    print(f"\n일반 능력 점수: {general_score}/{len(general_tests)} ({general_accuracy:.1%})")
    
    # 금융 능력 평가
    print("\n[금융 능력 테스트]")
    finance_score = 0
    for test in finance_tests:
        response = generate_response(model, tokenizer, test["question"], max_new_tokens=100)
        is_correct = any(kw in response for kw in test["keywords"])
        status = "✓" if is_correct else "✗"
        print(f"  {status} {test['question']} → {response[:80]}...")
        if is_correct:
            finance_score += 1
    
    finance_accuracy = finance_score / len(finance_tests)
    print(f"\n금융 능력 점수: {finance_score}/{len(finance_tests)} ({finance_accuracy:.1%})")
    
    # 결과 판정
    print("\n" + "=" * 50)
    if general_accuracy >= 0.75:
        print("일반 능력 유지됨 (재앙적 망각 최소화)")
    else:
        print("일반 능력 저하 (재앙적 망각 발생)")
    
    return {
        "general_score": general_accuracy,
        "finance_score": finance_accuracy
    }

# 평가 실행
results = evaluate_catastrophic_forgetting(model, tokenizer)
print("평가 함수 준비 완료")

재앙적 망각 평가

[일반 능력 테스트]
  ✗ 1 + 1 = ? → There are many ways to invest money in the US. The...
  ✓ 대한민국의 수도는? → 서울입니다. 서울은 대한민국의 수도이며, 행정, 경제, 문화의 중심지이다. 인구는 약 10...
  ✓ 물의 화학식은? → H2O.

```

So basically, the user wants to see the...
  ✓ 태양계에서 가장 큰 행성은? → 제 답변은 목성이다. 목성은 태양계에서 가장 큰 행성이다. 목성은 태양에서 5번째 행성으로...

일반 능력 점수: 3/4 (75.0%)

[금융 능력 테스트]
  ✓ PER이 낮다는 것은 무엇을 의미하나? → PER(Price Earnings Ratio, 주가수익비율)는 주가가 주당순이익(EPS, Earnings Per Share) 대비 얼마나 높게 ...
  ✓ 금리가 오르면 채권 가격은? → 채권의 현재 가치는 미래 현금 흐름(이자와 원금)의 현재 가치이다. 미래 현금 흐름의 현재 가치는 현재 가치와 동일한 이자율(할인율)을 사용하여...
  ✓ 분산투자의 목적은? → 분산투자는 투자 포트폴리오에 여러 종류의 자산을 포함시켜 위험을 최소화하는 전략이다. 위험을 줄이기 위해서는 포트폴리오에 포함되는 자산들의 상관...

금융 능력 점수: 3/3 (100.0%)

일반 능력 유지됨 (재앙적 망각 최소화)
평가 함수 준비 완료


## 12. 요약

### OSF 핵심 포인트

| 항목 | 설명 |
|------|------|
| **원리** | SVD로 가중치 분해 후 high-rank 부분 동결, low-rank 부분만 학습 |
| **장점** | 재앙적 망각 방지, 추가 파라미터 없음, 순차 학습에 최적화 |
| **핵심 파라미터** | `effective_rank` (보존할 rank, LoRA의 r과 반대 개념) |
| **권장 학습률** | 1e-5 ~ 1e-4 |
| **순차 학습 전략** | 태스크마다 effective_rank 점진적 증가 |

### OSF vs LoRA 비교

| 특성 | LoRA | OSF |
|------|------|-----|
| **rank 의미** | 학습 가능한 rank | 보존할 rank |
| **추가 파라미터** | A, B 행렬 추가 | 없음 (in-place) |
| **지속 학습** | 제한적 | 설계 목적 |
| **메모리** | 약간 증가 | 변화 없음 |

---

이 튜토리얼을 통해 HuggingFace PEFT의 OSF를 활용하여 재앙적 망각 없이 새로운 도메인 지식을 모델에 주입하는 방법을 학습했다.